In [43]:

import re
from os import listdir
from os.path import isfile, join

# 폴더내 pdf파일을 txt 파일로 변경
def hwp2txt (source_folder="minutes/hwp/", output_folder="minutes/text/") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    hwp_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    for hwp in hwp_files :
        hwp_filepath = source_folder + hwp
        # pdf 파일을 text로 변환
        print(hwp_filepath)
        hwp5txt <hwp_filepath>
   
            
hwp2txt()

SyntaxError: invalid syntax (<ipython-input-43-16bc7fca40a9>, line 14)

In [37]:
import olefile
f=olefile.OleFileIO("./hwp/20060112_20060228.hwp")
encoded_text=f.openstream('PrvText').read()
decoded_text=encoded_text.decode("UTF-16")
# print(decoded_text)
print(decoded_text)

금융통화위원회 의사록

2006년도 제2차 회의

1. 일    자     2006년 1월 12일 (목) 

2. 장    소     금융통화위원회 회의실

3. 출석위원     박    승  의 장 (총 재)
김 태 동  위 원 (의장직무대행)
이 성 태  위 원 (부총재)
김 종 창  위 원
강 문 수  위 원
이 덕 훈  위 원
이 성 남  위 원

4. 결석위원     없     음

5. 참 여 자     이 상 용  감사                 정 규 영  부총재보
               정 방 우  부총재보             박 재 환  부총재보
               김 수 명  부총재보             이 영 균  부총재보
               정 해 왕  금융경제연구원장     김 재 천  조사국장
               이 주 열  정책기획국장         김 수 호  금융시장국장
               이 광 주  국제국장             정 이 모  금융통화위원회실장
               이 광 준  공보실장             

6. 회의경과

〈의안 제2호 ― 통화정책방향〉

(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제2호 - ｢국내외 경제동향｣, 국제국장이 보고 제3호 - ｢외환․국제금융 동향｣, 그리고 금융시장국장이 보고 제4호 - ｢금융시장 동향｣에 대하여 보고하였음(보고내용: 별첨)

(２) 본회의에서는 의장이 ｢한국은행법｣ 제28조에 의거하여 의안 제2호 - ｢통화정책방향｣을 상정하였음

(３) 위원 토의내용 

   ｢외환․국제금융 동향｣과 관련하여, 일부 위원은 2002년부터 거의 예외 없이 연초에는 원화의 강세 현상이 나타나 왔다며 최근의 환율 하락도 이러한 계절적 현상이 거듭되고 있는 것으로 볼 수 있는지, 아니면 다른 특별한 원인에 기인한 것인지 물었으며, 이에 대해 관련부서에서는 연말연시에는 대체로 외환 공급이 많은 반면 수요는 


In [2]:
import os
import sys
import platform
import pandas as pd
import re
from datetime import datetime, timedelta

In [3]:
import requests
from bs4 import BeautifulSoup
import html2text
from datetime import datetime

In [3]:
def get_minutes_list(from_date='20050101'):
    prefix_addr = "https://www.bok.or.kr"
    from_date = datetime.strptime(from_date, '%Y%m%d')

    for pageIndex in range(1, 31):
        url = 'https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={}'.format(pageIndex)
        user_agent = 'Mozilla/5.0'
        headers ={'User-Agent' : user_agent}
        page = requests.get(url, headers=headers)

        try:
            soup = BeautifulSoup(page.content, 'html.parser')
            brdList = soup.find_all('span', class_='col m10 s10 x9 ctBx')
            datainfo = soup.find_all('div', class_='col s12 dataInfo')

            for post, data in zip(brdList, datainfo):
                guid = prefix_addr+post.a['href'][:-12]

                desPage = requests.get(guid)
                desSoup = BeautifulSoup(desPage.content, 'html.parser')   
                description = desSoup.find('div', class_='dbData').get_text().strip()
                if description.replace(' ','').find('통화정책방향') >= 0:
                    title = post.find('span', class_='titlesub').get_text().strip()
        
                    mdate = title[title.find(')(') + 2:-1]            
                    if mdate[-1] == '.':
                        mdate = mdate[:-1]
                    mdate = datetime.strptime(mdate, '%Y.%m.%d')

                    if mdate < from_date:
                        break

                    rdate = data.find('span', class_='date').get_text().strip()
                    rdate = datetime.strptime(rdate[3:], '%Y.%m.%d')    

                    get_minutes_file(guid, mdate, rdate)
        except:
            print("get url.content error and pass page{} it".format(pageIndex))

In [4]:
def get_minutes_file(page_addr, mdate, rdate):
    file_header = 'hwp/'
    prefix_addr = "http://bok.or.kr"

    page = requests.get(page_addr)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    try:
        links = soup.find('div', class_='addfile').find_all('a')

        for link in links:
            filename = link.get_text()
            filename = filename.replace('\r', '').replace('\t', '').replace('\n', '')

            if filename[-3:] == 'hwp':
                filename = mdate.strftime('%Y%m%d') + "_" + rdate.strftime('%Y%m%d')+ '.hwp'
                file_addr = prefix_addr + link["href"]
                file_res = requests.get(file_addr)
                filepath = file_header + filename

                with open(filepath, 'wb') as f:
                    f.write(file_res.content)

                print('save file name : ' + filename)
    except:
        print("get file failed and pass it")

In [5]:
get_minutes_list()

save file name : 20190830_20190917.hwp
save file name : 20190718_20190806.hwp
save file name : 20190531_20190618.hwp
save file name : 20190418_20190507.hwp
save file name : 20190228_20190319.hwp
save file name : 20190124_20190212.hwp
save file name : 20181130_20181218.hwp
save file name : 20181018_20181106.hwp
save file name : 20180831_20180918.hwp
save file name : 20180712_20180731.hwp
save file name : 20180524_20180612.hwp
save file name : 20180412_20180502.hwp
save file name : 20180227_20180320.hwp
save file name : 20180118_20180206.hwp
save file name : 20171130_20171219.hwp
save file name : 20170831_20170919.hwp
save file name : 20170713_20170801.hwp
save file name : 20170525_20170613.hwp
save file name : 20170413_20170502.hwp
save file name : 20170223_20170314.hwp
save file name : 20170113_20170131.hwp
save file name : 20161215_20170103.hwp
save file name : 20161111_20161129.hwp
save file name : 20161013_20161101.hwp
save file name : 20160909_20160927.hwp
save file name : 20160811

In [6]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [15]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [16]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text) :
    mpck = MPCK()
    tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(tokens)
                
    return tokens+ ngrams

In [39]:
with open("./20060112_20060228.txt","r",encoding="UTF-8") as f:
    txt=f.read()
    
txt

'\ufeff금융통화위원회 의사록\n\n2006년도 제2차 회의\n\n1. 일    자     2006년 1월 12일 (목) \n\n2. 장    소     금융통화위원회 회의실\n\n3. 출석위원     박    승  의 장 (총 재)\n김 태 동  위 원 (의장직무대행)\n이 성 태  위 원 (부총재)\n김 종 창  위 원\n강 문 수  위 원\n이 덕 훈  위 원\n이 성 남  위 원\n\n4. 결석위원     없     음\n\n5. 참 여 자    이 상 용  감사                 정 규 영  부총재보\n               정 방 우  부총재보             박 재 환  부총재보\n               김 수 명  부총재보             이 영 균  부총재보\n               정 해 왕  금융경제연구원장     김 재 천  조사국장\n               이 주 열  정책기획국장         김 수 호  금융시장국장\n               이 광 주  국제국장             정 이 모  금융통화위원회실장\n               이 광 준  공보실장             \n\n6. 회의경과\n\n〈의안 제2호 ― 통화정책방향〉\n\n(１) 전일 개최된 동향보고회의에서 조사국장이 보고 제2호 - ｢국내외 경제동향｣, 국제국장이 보고 제3호 - ｢외환․국제금융 동향｣, 그리고 금융시장국장이 보고 제4호 - ｢금융시장 동향｣에 대하여 보고하였음(보고내용: 별첨)\n\n(２) 본회의에서는 의장이 ｢한국은행법｣ 제28조에 의거하여 의안 제2호 - ｢통화정책방향｣을 상정하였음\n\n(３) 위원 토의내용 \n\n   ｢외환․국제금융 동향｣과 관련하여, 일부 위원은 2002년부터 거의 예외 없이 연초에는 원화의 강세 현상이 나타나 왔다며 최근의 환율 하락도 이러한 계절적 현상이 거듭되고 있는 것으로 볼 수 있는지, 아니면 다른 특별한 원인에 기인한 것인지 물었으며, 이에 대해 관련부서에서는 

In [23]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join

    
def preprocessing(source_folder="./minutes/hwp/", output_file="./minutes/minutes_preprocessed_hwp.csv") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    txt_files.sort()
    df = pd.DataFrame(columns=['date', 'minutes'])
    df["Economic Situation"] = ""
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df["Monetary Policy"] = ""
    df["Participant Views"] = ""
    df["Government View"] = ""

    df["Economic Situation count"] = ""
    df["Foreign Currency count"] = ""
    df["Financial Markets count"] = ""
    df["Monetary Policy count"] = ""
    df["Participant Views count"] = ""
    df["Government View count"] = ""

    df["ngram"] = ""
    

    for txt_file in txt_files :
        with open(source_folder + txt_file, 'r') as f :
            txt = f.read()
            sections, section_texts = preprocess_minutes(txt)
            
            print(txt_file)
            
            ngrams = []
            for section_text in section_texts[1:3] :
                for sentence in section_text :
                    ngrams.append(','.join(text2ngram(sentence)))
                
            #print('@@@'.join(ngrams))
            #break

            df.loc[len(df)] = [txt_file.split('_')[0],
                               txt,
                               '@@@'.join(section_texts[0]),
                               '@@@'.join(section_texts[1]),
                               '@@@'.join(section_texts[2]),
                               '@@@'.join(section_texts[3]),
                               '@@@'.join(section_texts[4]),
                               '@@@'.join(section_texts[5]),
                               len(section_texts[0]),
                               len(section_texts[1]),
                               len(section_texts[2]),
                               len(section_texts[3]),
                               len(section_texts[4]),
                               len(section_texts[5]),
                               '@@@'.join(ngrams)]
    
    df.to_csv(output_file, index=False)
        
    print(df.shape)
preprocessing()

UnicodeDecodeError: 'cp949' codec can't decode byte 0xb1 in position 5: illegal multibyte sequence

In [18]:
import pandas as pd
df = pd.read_csv("./minutes/minutes_preprocessed.csv")
df['ngram'][0]

nan

In [19]:
df

,date,minutes,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,ngram
0,20060112,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
1,20060209,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
2,20060309,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
3,20060407,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
4,20060511,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,20190228,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 중국의 완화적인 통화·재정정책과 관련하여, 중국 고위당국자 간 신용 ...",NaN,NaN,일부 위원은 금융시장 가격변수를 활용하여 경기국면 전환 가능성을 분석할 경우 과거...,일부 위원은 국내경제와 세계경제의 성장 둔화세가 예상된 범위 내에서 진행 되는 가...,NaN,357,0,0,8,129,0,NaN
150,20190418,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 금번 관련부서의 GDP성장률 수정 전망치인 2.5%에 동의한다고 언급...,일부 위원은 금년 들어 은행을 통한 외화자금 유입과 함께 명목실효환율이 완만한 하...,NaN,일부 위원은 통화정책 파급효과 분석 시 기준금리 조정 시기별로 금융·경제 여건이 ...,"일부 위원은 이번 회의에서는 기준금리를 현 수준에서 유지하고, 향후 미국을 포함한...",NaN,76,31,0,31,139,0,"들/VV,은행/NNG,통하/VV,외화자금/NNG,유입/NNG,함께/MAG,명목/NN..."
151,20190531,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 유로지역의 노동시장이 지표상으로 양호한 상황을 지속하고 있 는데, 그...",NaN,NaN,일부 위원은 경제현상의 틀이 바뀌는 국면에서는 기존에 정립된 이론으로 설 명하기 ...,일부 위원은 기대인플레이션 약화에 대한 우려는 있으나 여러 구조적 문제점 과 현재...,NaN,368,0,0,38,128,0,NaN
152,20190718,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 관련부서에서 경제 분석 및 전망시 활용하는 일부 거시계량모형 이 재정...,일부 위원은 최근의 외환수급 동향을 볼 때 외은지점을 중심으로 금융거래가 확대되고...,NaN,일부 위원은 일반적으로 중앙은행은 최종대부자(lender of last resort...,일부 위원은 기준금리를 현재의 1.75%에서 1.50%로 인하하는 것이 적절하다 ...,NaN,50,59,0,40,132,0,"외환/NNG,수급/NNG,동향/NNG,보/VV,때/NNG,외/VA,지점/NNG,중심..."


In [ ]:
df[['date','Economic Situation count', 'Foreign Currency count','Financial Markets count','Monetary Policy count','Participant Views count','Government View count']].to_csv("./results/minutes_daily_count.csv", index = False)